### Notebook example with python + elastic


##### First we install and import necessary libraries and instatiate a client to connect to our Elastic instance


In [ ]:
! pip install pandas elasticsearch

In [8]:
# Import necessary libraries
import pandas as pd  # For data manipulation with DataFrames
from getpass import getpass  # For securely getting user input (passwords)
from elasticsearch import Elasticsearch, helpers  # For interacting with Elasticsearch

In [17]:
#Connect to the elastic cloud server
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Elastic API Key: ")

# Create an Elasticsearch client using the provided credentials
es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,  # cloud id can be found under deployment management
    api_key=ELASTIC_API_KEY, # your username and password for connecting to elastic, found under Deplouments - Security
)

##### Getting the data and creating the index

In [14]:
# Read data from a CSV file into a DataFrame
# The Repair Cafe International dataset can be downloaded at https://openrepair.org/open-data/downloads/
repair_data_df = pd.read_csv('OpenRepairData_v0.3_RepairCafeInt_202309.csv')

# FiYll missing values in the DataFrame with 0
repair_data_df = repair_data_df.fillna(0)

In [19]:
repair_data_df.head()

,id,data_provider,country,partner_product_category,product_category,product_category_id,brand,year_of_manufacture,product_age,repair_status,repair_barrier_if_end_of_life,group_identifier,event_date,problem
0,rcint_55417,Repair Café International,GBR,Household appliances electric ~ Carpet cleaner,Small kitchen item,29,Vax,2010.0,13.0,End of life,Item too worn out,Weymouth,2023-09-09,Engine motor burnt out. Not working.
1,rcint_55422,Repair Café International,NLD,Tools electric ~ Motor,Power tool,24,DeWert,1994.0,29.0,Repairable,0,Capelle aan den IJssel,2023-09-28,trafo defect. werkt niet meer. advies gegeven...
2,rcint_55432,Repair Café International,NLD,Household appliances electric ~ Flashlight / e...,Decorative or safety lights,3,Unknown,0.0,0.0,End of life,0,Laren (Noord-Holland),2017-04-01,Zwak licht. Misschien te ver ontladen. Accu ...
3,rcint_55433,Repair Café International,NLD,Household appliances electric ~ Lamp / lighting,Lamp,15,Unknown,0.0,0.0,End of life,0,Delft-Delfshove,2019-10-05,Nee.
4,rcint_55434,Repair Café International,NLD,Household appliances electric ~ Shaver / Razor,Hair & beauty item,9,Philips,0.0,0.0,End of life,0,Delft,2018-09-01,plastic afgebroken. Defect. Niet te repareren.


In [18]:
index_name = 'repair'

# Create the Elasticsearch index with the specified name
es.indices.create(index=index_name)

# Define a function to convert DataFrame rows to Elasticsearch documents
def df_to_doc(df, name_of_index):
    for index, document in df.iterrows():
        yield dict(_index=name_of_index, _id=f"{document['id']}", _source=document.to_dict())

# Use the Elasticsearch helpers.bulk() method to index the DataFrame data into Elasticsearch
load = helpers.bulk(es, df_to_doc(repair_data_df, index_name))

##### Simple Query examples

In [25]:
query={
        "match": {
            "problem": "engine"
        }
    }

#Run a simple query, for example looking for problems with the engine
response = es.search(index=index_name, query=query)

print("We get back {total} results, here are the top ones:".format(total=response["hits"]['total']['value']))
for hit in response["hits"]["hits"]:
    print(hit['_source']['problem'])

We get back 19 results, here are the top ones:
blocked engine.
Engine malfunction. Doesn't work.
Clock engine was stuck.
Engine is burnt. Doesn't work.
Engine seized. Doesn't turn over.
Engine runs, blades don't turn.
Engine motor burnt out. Not working.
disconnect between engine and shaft. doesn't work.
engine shaft off-kilter. makes friction noise.
yes - wiring has completely corroded. problem with the engine.
